# Capture the Flag (RL - Policy Gradient)

- Seung Hyun Kim
- skim449@illinois.edu

## Implementation Details

- Actor-critic
- On Policy

### Sampling
- [ ] Mini-batch to update 'average' gradient
- [ ] Experience Replay for Random Sampling
- [ ] Importance Sampling
    
### Deterministic Policy Gradient
- [ ] DDPG
- [ ] MADDPG

### Stability and Reducing Variance
- [x] Gradient clipping
- [ ] Normalized Reward/Advantage
- [ ] Target Network
- [ ] TRPO
- [ ] PPO

### Multiprocessing
- [ ] Synchronous Training (A2C)
- [x] Asynchronous Training (A3C)

### Applied Training Methods:
- [ ] Self-play
- [ ] Batch Policy

## Notes

- This notebook includes:
    - Building the structure of policy driven network.
    - Training with/without render
    - Saver that save model and weights to ./model directory
    - Writer that will record some necessary datas to ./logs

- This notebook does not include:
    - Simulation with RL policy
        - The simulation can be done using policy_RL.py
    - cap_test.py is changed appropriately.
    
## References :
- https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb (source)
- https://www.youtube.com/watch?v=PDbXPBwOavc
- https://github.com/lilianweng/deep-reinforcement-learning-gym/blob/master/playground/policies/actor_critic.py (source)
- https://github.com/spro/practical-pytorch/blob/master/reinforce-gridworld/reinforce-gridworld.ipynb

## TODO:

- Research on '_bootstrap_' instead of end-reward
- Add global step
- Think about adding discont to advantage
- Normalize reward?
- Record method in network

In [1]:
TRAIN_NAME='A3C_lstm_try'
LOG_PATH='./logs/'+TRAIN_NAME
MODEL_PATH='./model/' + TRAIN_NAME

In [2]:
import os
import configparser
#from tqdm import tqdm

import signal
import threading
import multiprocessing

import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.layers as layers
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
%matplotlib inline

import time
import gym
import gym_cap
import gym_cap.envs.const as CONST
import numpy as np
import random
import math

# the modules that you can use to generate the policy. 
import policy.random
import policy.roomba
import policy.policy_RL
import policy.zeros

# Data Processing Module
from utility.dataModule import one_hot_encoder_v2 as one_hot_encoder
from utility.utils import MovingAverage as MA
from utility.utils import discount_rewards
from utility.buffer import Trajectory, Trajectory_buffer

from network.ActorCritic_lstm import ActorCritic as Network

%load_ext autoreload
%autoreload 2

/Users/skim0119/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Hyperparameters

In [3]:
# Importing global configuration
config = configparser.ConfigParser()
config.read('config.ini')

## Environment
action_space = config.getint('DEFAULT','ACTION_SPACE')
n_agent = config.getint('DEFAULT','NUM_AGENT')
map_size = config.getint('DEFAULT','MAP_SIZE')
vision_range = config.getint('DEFAULT','VISION_RANGE')

## Training
total_episodes = config.getint('TRAINING','TOTAL_EPISODES')
max_ep = config.getint('TRAINING','MAX_STEP')
critic_beta = config.getfloat('TRAINING', 'CRITIC_BETA')
entropy_beta = config.getfloat('TRAINING', 'ENTROPY_BETA')
gamma = config.getfloat('TRAINING', 'DISCOUNT_RATE')

decay_lr = config.getboolean('TRAINING','DECAYING_LR')
lr_a = 1e-3 # config.getfloat('TRAINING','LR_ACTOR')
lr_c = 1e-2 # config.getfloat('TRAINING','LR_CRITIC')

## Save/Summary
save_network_frequency = config.getint('TRAINING','SAVE_NETWORK_FREQ')
save_stat_frequency = config.getint('TRAINING','SAVE_STATISTICS_FREQ')
moving_average_step = config.getint('TRAINING','MOVING_AVERAGE_SIZE')

## GPU
gpu_capacity = config.getfloat('GPU_CONFIG','GPU_CAPACITY')
gpu_allowgrow = config.getboolean('GPU_CONFIG', 'GPU_ALLOWGROW')

In [4]:
# Local configuration parameters
po_transition = 100000 # Partial observable
#serial_length = 8

# Env Settings
n_channel = 11
vision_dx, vision_dy = 2*vision_range+1, 2*vision_range+1
in_size = [None,vision_dx,vision_dy,n_channel]
nenv = 8 #(int) (multiprocessing.cpu_count())

# Asynch Settings
global_scope = 'global'

## Environment Setting

In [5]:
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
#Create a directory to save episode playback gifs to
if not os.path.exists(LOG_PATH):
    os.makedirs(LOG_PATH)

## A3C Network Structure

![Network Structure](https://cdn-images-1.medium.com/max/1600/1*YtnGhtSAMnnHSL8PvS7t_w.png)

- Network is given in network.ActorCritic

## Environments

![Environment Interaction Diagram](https://cdn-images-1.medium.com/max/1600/1*Hzql_1t0-wwDxiz0C97AcQ.png)

In [6]:
global_rewards = MA(moving_average_step)
global_ep_rewards = MA(moving_average_step)
global_length = MA(moving_average_step)
global_succeed = MA(moving_average_step)
global_episodes = 0

# Launch the sessi

sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
progbar = tf.keras.utils.Progbar(total_episodes,interval=1)

## Environment Unit

In [7]:
class Environment(threading.Thread):
    def __init__(self, name, global_network, sess, global_step, coord):
        super(Environment, self).__init__()
        # Initialize Environment worker
        self.env = gym.make("cap-v0").unwrapped
        self.name = name
        self.global_network = global_network
        self.sess = sess
        self.global_step = global_step
        self.coord = coord
        
        # Create AC Network for Worker
        self.local_network = Network(in_size=in_size,
                                     action_size=action_space,
                                     lr_actor=lr_a,
                                     lr_critic=lr_c,
                                     scope=self.name,
                                     #grad_clip_norm=10.0,
                                     global_step=global_step,
                                     entropy_beta=entropy_beta,
                                     sess=sess,
                                     global_network=global_ac)

        

        
    def run(self, saver, writer):
        """Override Thread.run

        Note:
            Loop to run rollout
            Include summarizing and save
        """
        self.saver = saver
        self.writer = writer
        
        global global_rewards, global_ep_rewards, global_length, global_succeed, global_episodes
        total_step = 0
        while not self.coord.should_stop() and global_episodes < total_episodes:
            self.rollout(init_step=total_step)
            
    def get_action(self, states, rnn_states):
        """Run graph to get action

        Args:
            state (list): list of state for each agent
            rnn_states (list): list of rnn inputs for each agent

        Returns:
            action (list) : The action for each egent
            values (list) : The value for each action for each agent
            rnn_next (list) : List of next rnn state for each agent

        Note:
            If rnn_states=None, get action without rnn_states.
        """
        actions, values, final_states = self.local_network.feed_forward(
            state=np.expand_dims(states,axis=1),
            rnn_init_state=rnn_states,
            seq_len=[1]*len(states)
            )
            
        self.feed_dict = {self.local_network.state_input_: np.expand_dims(states, axis=1),
                          self.local_network.rnn_init_states_: self.local_network.get_lstm_initial(4),
                          self.local_network.seq_len_: [1]*n_agent}
        return actions, values, final_states
            
    def rollout(self, init_step=0):
        global global_episodes
        total_step = init_step
        with self.sess.as_default(), self.sess.graph.as_default():
            # Initialize run
            s0 = self.env.reset(map_size=map_size,
                                #seed=int(global_episodes % 10) + 10,
                                policy_red=policy.zeros.PolicyGen(self.env.get_map,
                                                                  self.env.get_team_red))
            if po_transition < global_episodes:
                s0 = one_hot_encoder(s0, self.env.get_team_blue, vision_range)
            else:
                s0 = one_hot_encoder(self.env._env, self.env.get_team_blue, vision_range)

            # parameters 
            ep_r = 0 # Episodic Reward
            prev_r = 0
            step = 0
            d = False
            
            # Trajectory Buffers
            trajs = [Trajectory(depth=6) for _ in range(n_agent)]

            # RNN Initializer
            self.rnn_states = self.local_network.get_lstm_initial(n_agent)

            # Bootstrap
            a1, v1, final_states = self.get_action(s0, self.rnn_states)
            is_alive = [ag.isAlive for ag in self.env.get_team_blue]
            
            while step <= max_ep and not d:
                a, v0 = a1, v1
                was_alive = is_alive
                self.rnn_states = np.asarray(final_states)

                s1, rc, d, _ = self.env.step(a)
                if po_transition < global_episodes:
                    s1 = one_hot_encoder(s1, self.env.get_team_blue, vision_range)
                else:
                    s1 = one_hot_encoder(self.env._env, self.env.get_team_blue, vision_range)
                is_alive = [ag.isAlive for ag in self.env.get_team_blue]
                r = rc - prev_r - 0.5
                
                if step == max_ep and d == False:
                    r = -100
                    rc = -100
                    d = True

                r /= 100.0
                ep_r += r

                if d:
                    v1 = [0.0 for _ in range(n_agent)]
                else:
                    a1, v1, final_states = self.get_action(s1, self.rnn_states)

                # push to buffer
                for idx, agent in enumerate(self.env.get_team_blue):
                    if was_alive[idx]:
                        trajs[idx].append([s0[idx],
                                           a[idx],
                                           r,
                                           v0[idx],
                                           0,
                                           self.rnn_states[:,idx,:]
                                          ])

                # Iteration
                prev_r = rc
                total_step += 1
                step += 1
                s0 = s1
                
                # self.env.render(mode='fast')
            print(self.env.blue_win, ep_r, step)
            #self.local_network.pull_global()

## Run

In [8]:
def initialize_uninitialized_vars(sess):
    from itertools import compress
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([~(tf.is_variable_initialized(var)) \
                                   for var in global_vars])
    not_initialized_vars = list(compress(global_vars, is_not_initialized))

    if len(not_initialized_vars):
        sess.run(tf.variables_initializer(not_initialized_vars))

In [9]:
coord = tf.train.Coordinator()
# Global Network
global_step = tf.Variable(0, trainable=False, name='global_step')
#global_step_next = tf.assign_add(global_step, 1)
global_ac = Network(in_size=in_size,
                    action_size=action_space,
                    scope=global_scope,
                    sess=sess,
                    global_step=global_step)

# Local workers
workers = []
# loop for each workers

for idx in range(nenv):#tqdm(range(nenv), ncols=65, desc="Process Initiate"):
    name = 'W_%i' % idx
    workers.append(Environment(name, global_ac, sess, global_step=global_step, coord=coord))

writer = tf.summary.FileWriter(LOG_PATH, sess.graph)

    
ckpt = tf.train.get_checkpoint_state(MODEL_PATH)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    #tf.reset_default_graph()
    #saver.restore(sess, ckpt.model_checkpoint_path)
    saver = tf.train.Saver(max_to_keep=3)
    #saver = tf.train.import_meta_graph(ckpt.model_checkpoint_path+'.meta', clear_devices=True)
    saver.restore(sess, ckpt.model_checkpoint_path)
    #initialize_uninitialized_vars(sess)
    print("Load Model : ", ckpt.model_checkpoint_path)
else:
    raise ValueError

worker_threads = []
global_episodes = 0#sess.run(global_step)

worker = workers[0]
worker.run(saver, writer)

/Users/skim0119/anaconda3/lib/python3.6/site-packages/gym-0.10.5-py3.6.egg/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


INFO:tensorflow:Restoring parameters from ./model/A3C_lstm_try/ctf_policy.ckpt-10000
Load Model :  ./model/A3C_lstm_try/ctf_policy.ckpt-10000
False -1.8750000000000007 151
True 0.945 11
False -2.000000000000001 151
False -1.7500000000000004 151
True 0.74 52
True 0.965 7
True 0.875 25
True 0.92 16
False -1.7500000000000004 151
False -2.000000000000001 151
False -1.8750000000000007 151
False -1.7500000000000004 151
True 0.9 20
False -1.7500000000000004 151
False -1.7500000000000004 151
False -1.8750000000000004 151
False -1.6250000000000004 151
False -1.7500000000000004 151
False -1.7500000000000004 151
False -2.0000000000000004 151
False -1.7500000000000004 151
False -1.7500000000000004 151
False -1.8750000000000004 151
False -2.0000000000000004 151
False -1.7500000000000007 151
False -1.8750000000000004 151
True 0.925 15
False -2.124999999999998 151
False -1.7500000000000004 151
False -1.8750000000000007 151
False -1.7500000000000004 151
False -1.7500000000000004 151
False -1.875000000

KeyboardInterrupt: 